In [1]:
import andes
from andes.interop.pandapower import to_pandapower, make_GSF
import pandapower as pp

from numpy import pi
from andes.shared import pd, rad2deg, deg2rad

import numpy as np
import andes.interop.pandapower as adpp

import pandas as pd

In [2]:
# !andes run '/Users/jinningwang/Documents/work/andes/jwang/case/case2000.m' --convert

In [3]:
print(andes.__version__)
print(pp.__version__)

1.7.1.post212.dev0+g3dd1423c
2.8.0


In [4]:
# ss0 is used for PP conversion

case2k = '/Users/jinningwang/Documents/work/andes/jwang/case/case2000.xlsx'
case14 = andes.get_case('ieee14/ieee14_ac8b.xlsx')
ssa = andes.load(case2k,
                 setup=True,
                 no_output=True,
                 default_config=False)

Generating code for 1 models on 8 processes.


In [5]:
ssp = to_pandapower(ssa, verify=False)

In [6]:
gsf = make_GSF(ssp)